In [12]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "infiniteMemory1!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.I(html.H2('Tanner Hunt'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(
        className = 'buttonRow',
        style={'display' : 'flex'},
        children = [
            dcc.RadioItems(['Water Rescue Dogs', 
                            'Mountain or Wilderness Rescue Dogs', 
                            'Disaster or Individual Tracking Dogs', 
                            'None'],
                id='filter_radio_buttons', 
                inline=True,
                value='None',
                labelStyle={'padding-right' : 10}
                       )
        ]

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        page_current=0,
        page_size=10,
        page_action='custom',
        row_selectable = "single"

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
# This callback handles pagination of the datatable
@app.callback(
    Output('datatable-id', 'data'),
    Input('datatable-id', 'page_current'),
    Input('datatable-id', 'page_size'),
    Input('filter_radio_buttons', 'value')
)
def update_table(page_current, page_size, selectedFilter):
    # Filter dogs by job
    if selectedFilter == 'Water Rescue Dogs':
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog',
                                               'breed' : {'$in' : ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                                               'sex_upon_outcome' : 'Intact Female',
                                               'age_upon_outcome_in_weeks' : {'$gte' : 26},
                                               'age_upon_outcome_in_weeks' : {'$lte' : 156}}))
        df.drop(columns=['_id'],inplace=True)
    elif selectedFilter == 'Mountain or Wilderness Rescue Dogs':
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog',
                                       'breed' : {'$in' : ['German Shepherd', 'Alaskan', 'Malamute', 'Old English', 'Sheepdog', 'Siberian Husky', 'Rottweiler']},
                                       'sex_upon_outcome' : 'Intact Male',
                                       'age_upon_outcome_in_weeks' : {'$gte' : 26},
                                       'age_upon_outcome_in_weeks' : {'$lte' : 156}}))
        df.drop(columns=['_id'],inplace=True)
    elif selectedFilter == 'Disaster or Individual Tracking Dogs':
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog',
                                               'breed' : {'$in' : ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                               'sex_upon_outcome' : 'Intact Male',
                                               'age_upon_outcome_in_weeks' : {'$gte' : 20},
                                               'age_upon_outcome_in_weeks' : {'$lte' : 300}}))
        df.drop(columns=['_id'],inplace=True)
    else: # None filter; show all dogs
        df = pd.DataFrame.from_records(db.read({}))
        df.drop(columns=['_id'],inplace=True)
    
    return df.iloc[page_current*page_size:(page_current+1)*page_size
                  ].to_dict('records')



# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('filter_radio_buttons', 'value')])
def update_graphs(viewData, selectedFilter):
    pass
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    return [
        dcc.Graph(            
            figure = px.pie(viewData, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # This callback is being made multiple times before selected_columns is instantiated.
    # Who knows why.  This clause prevents the compiler from throwing errors
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    dff = pd.DataFrame.from_dict(viewData)
    # The callback is being made multiple times whenever the datatable is changed
    # but the dataframe is empty for several of those.  IDK why this is, 
    # but this clause stops the compiler from throwing errors for accessing
    # an empty dataframe
    if dff.shape[1] == 0:
        return []

    # Because we only allow single row selection, the list can
    # be converted to a row index here
    if index == []:
        row = 0
    else:
        row = index[0]

        # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height' : '500px'},
              center=[30.75, -97.48], zoom=10, children=[
                  dl.TileLayer(id='base-layer-id'),
                  # Marker with tool tip and popup
                  # Column 13 and 14 define the grid-coordinates for
                  # the map
                  # column 4 defines the breed for the animal
                  # Column 9 defines the name of the animal
                  dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                           children=[
                               dl.Tooltip(dff.iloc[row,4]),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(dff.iloc[row, 9])
                               ])
                       ])
              ])
    ]


app.run_server(debug=True)

Dash app running on http://127.0.0.1:14801/
